In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, GRU, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import resample
from sklearn.metrics import classification_report
import re
import nltk
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")

2024-08-24 23:28:11.484053: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-24 23:28:11.484720: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-24 23:28:11.488244: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-24 23:28:11.496417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-24 23:28:11.509188: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
df = pd.read_csv("SPAM text message 20170820 - Data.csv")

In [3]:
df["Category"].value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

## Downsampling

In [4]:
ham = df[df.Category == "ham"]
spam = df[df.Category == 'spam']

In [5]:
ham

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
6,ham,Even my brother is not like to speak with me. ...
...,...,...
5565,ham,Huh y lei...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [6]:
ham_ds = resample(ham,
             replace=True,  
             n_samples=len(spam),  
             random_state=42)

In [7]:
data = pd.concat([ham_ds, spam])

In [8]:
data['Category'].value_counts()

Category
ham     747
spam    747
Name: count, dtype: int64

In [9]:
def clean_text(text):
    # Remove all alpha-numeric characters
    text = re.sub(r'\W', ' ', text)  
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)  
    text = text.strip()  # Remove leading and trailing spaces
    return text

In [10]:

data['Message'].apply(clean_text)

1014    Just buy a pizza Meat lovers or supreme U get ...
4364    Evry Emotion dsn t hav Words Evry Wish dsn t h...
3570                       She s fine Sends her greetings
546                                Then any special there
5121                             Oops I did have it lt gt
                              ...                        
5537    Want explicit SEX in 30 secs Ring 02073162414 ...
5540    ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547    Had your contract mobile 11 Mnths Latest Motor...
5566    REMINDER FROM O2 To get 2 50 pounds free call ...
5567    This is the 2nd time we have tried 2 contact u...
Name: Message, Length: 1494, dtype: object

In [11]:
data['message_length'] = data['Message'].apply(len)

In [12]:
data

,Category,Message,message_length
1014,ham,Just buy a pizza. Meat lovers or supreme. U ge...,56
4364,ham,Evry Emotion dsn't hav Words.Evry Wish dsn't h...,180
3570,ham,She's fine. Sends her greetings,31
546,ham,Then any special there?,23
5121,ham,"Oops I did have it, &lt;#&gt; ?",32
...,...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...,90
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...,158
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...,160
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...,147


In [13]:
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['Category'])
data.drop("Category",axis=1,inplace=True)

In [14]:
X = data['Message']
y = data['label']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

## Preprocessing text data

In [15]:
tokenizer = Tokenizer(num_words=500, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

In [16]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [17]:
len(X_train),len(X_test)

(1195, 299)

In [18]:
maxlen = 50
X_train_padded = pad_sequences(X_train_sequences, padding='post', truncating='post', maxlen=maxlen)
X_test_padded = pad_sequences(X_test_sequences, padding='post', truncating='post', maxlen=maxlen)


## Training Model

In [19]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=maxlen))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)))
model.add(Bidirectional(GRU(32, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(1, activation='sigmoid'))

I0000 00:00:1724522312.050167   17369 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-24 23:28:32.050914: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [20]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
history = model.fit(X_train_padded, y_train, epochs=25, batch_size=64, validation_data=(X_test_padded, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.5825 - loss: 0.8368 - val_accuracy: 0.7993 - val_loss: 0.6609
Epoch 2/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7739 - loss: 0.6501 - val_accuracy: 0.8027 - val_loss: 0.6318
Epoch 3/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8046 - loss: 0.7465 - val_accuracy: 0.8763 - val_loss: 0.6072
Epoch 4/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8276 - loss: 0.5418 - val_accuracy: 0.9231 - val_loss: 0.5695
Epoch 5/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9168 - loss: 0.4619 - val_accuracy: 0.9264 - val_loss: 0.5181
Epoch 6/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9322 - loss: 0.3563 - val_accuracy: 0.9231 - val_loss: 0.4496
Epoch 7/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9526 - loss: 1.6550 - val_accuracy: 0.9231 - val_loss: 0.4238
Epoch 8/25
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9404 - loss: 1.5184 - val_accuracy: 0.9064 - v

In [22]:
y_pred = (model.predict(X_test_padded) > 0.5).astype("int32")

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step


In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.97      0.91       139
           1       0.97      0.85      0.91       160

    accuracy                           0.91       299
   macro avg       0.91      0.91      0.91       299
weighted avg       0.91      0.91      0.91       299



In [24]:
predicted = label_encoder.inverse_transform(y_pred)

In [25]:
X_test = pd.DataFrame(X_test)

In [26]:
X_test["predicted"] = predicted

In [30]:
X_test['True'] = label_encoder.inverse_transform(y_test)

In [31]:
X_test

,Message,predicted,True
1126,For taking part in our mobile survey yesterday...,spam,spam
3998,Bored housewives! Chat n date now! 0871750.77....,spam,spam
3883,"Short But Cute: ""Be a good person, but dont tr...",ham,ham
4053,"Ya ok, then had dinner?",ham,ham
1272,If you still havent collected the dough pls le...,ham,ham
...,...,...,...
4243,Show ur colours! Euro 2004 2-4-1 Offer! Get an...,spam,spam
3412,Only once then after ill obey all yours.,ham,ham
3272,You have 1 new voicemail. Please call 08719181...,spam,spam
4914,"Goal! Arsenal 4 (Henry, 7 v Liverpool 2 Henry ...",spam,spam
